In [1]:
from optimization_algorithms_tools.algorithms import SimulatedAnnealing
from optimization_algorithms_tools.problems import TSP
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
import folium
import folium.plugins

In [2]:
# load list of all walmart locations on Ontario
wal_df = pd.read_csv("../search_optimization_tools/problems/samples/TSP/Walmart_ON.csv")

In [3]:
# city to region mapping from wikipedia
cityToRegion = {
    'Toronto': 'Toronto',
    'Ajax': 'Durham Region',
    'Clarington': 'Durham Region',
    'Brock': 'Durham Region',
    'Oshawa': 'Durham Region',
    'Pickering': 'Durham Region',
    'Scugog': 'Durham Region',
    'Uxbridge': 'Durham Region',
    'Whitby': 'Durham Region',
    'Burlington': 'Halton Region',
    'Halton Hills': 'Halton Region',
    'Milton': 'Halton Region',
    'Oakville': 'Halton Region',
    'Brampton': 'Peel Region',
    'Caledon': 'Peel Region',
    'Mississauga': 'Peel Region',
    'Aurora': 'York Region',
    'East Gwillimbury': 'York Region',
    'Georgina': 'York Region',
    'King': 'York Region',
    'Markham': 'York Region',
    'Newmarket': 'York Region',
    'Richmond Hill': 'York Region',
    'Vaughan': 'York Region',
    'Whitchurch-Stouffville': 'York Region',
    'Mono': 'Dufferin County',
    'Orangeville': 'Dufferin County',
    'Bradford West Gwillimbury': 'Simcoe County',
    'New Tecumseth': 'Simcoe County'
}

In [4]:
# selecting cities that are in Durham Region, York Region, or Toronto
cities_list = [city for city, region in cityToRegion.items() if city in wal_df.city.unique() and region in ['Durham Region', 'York Region', 'Toronto']]
print(cities_list)

['Toronto', 'Ajax', 'Oshawa', 'Pickering', 'Uxbridge', 'Whitby', 'Aurora', 'Markham', 'Newmarket', 'Richmond Hill', 'Vaughan']


In [5]:
# select walmarts thar are in the above list and is Supercentre
gta_part = wal_df[wal_df.store_number.str.startswith('Walmart Supercentre') & wal_df.city.isin(cities_list)].reset_index(drop=True)
wal_gta_count = gta_part.shape[0]
gta_part

,latitude,longitude,store_number,phone,address,city,Province,postal_code
0,43.866158,-79.013226,Walmart Supercentre; #3001,(905) 426-6160,270 Kingston Rd E RR #1,Ajax,ON,L1S 4S7
1,44.015010,-79.411176,Walmart Supercentre; #5778,(905) 841-0300,135 First Commerce Dr,Aurora,ON,L4G 0G2
2,43.871301,-79.215267,Walmart Supercentre; #1109,(905) 472-9582,500 Copper Creek Dr,Markham,ON,L6B 0S1
3,43.867108,-79.290345,Walmart Supercentre; #3053,(905) 477-6060,5000 Hwy-7 Unit Y006A,Markham,ON,L3R 4M9
4,44.066859,-79.484682,Walmart Supercentre; #3062,(905) 853-8811,17940 Yonge S,Newmarket,ON,L3Y 8S4
5,43.907599,-78.814547,Walmart Supercentre; #1153,(905) 579-3325,1300 King St E,Oshawa,ON,L1H 8J4
6,43.942780,-78.847706,Walmart Supercentre; #3161,(905) 404-6581,1471 Harmony Rd,Oshawa,ON,L1H 7K5
7,43.880690,-78.881540,Walmart Supercentre; #1056,(905) 438-1400,680 Laval Dr,Oshawa,ON,L1J 0B5
8,43.843959,-79.068945,Walmart Supercentre; #3186,(905) 619-9588,1899 Brock Rd Unit #1,Pickering,ON,L1V 4H7
9,43.877256,-79.409737,Walmart Supercentre; #3195,(905) 737-3457,1070 Major Mackenzie Dr E,Richmond Hill,ON,L4S 1P3


In [6]:
# get the lat and long locations of the above set of Walmart's and create the graph of roads that connects them
# that is within 42KM  
gta_part_loc = gta_part[['latitude', 'longitude']]

G = ox.graph_from_point(tuple(gta_part_loc.mean().to_list()), dist=42000, dist_type='network',
                        network_type='drive', clean_periphery=True, simplify=True, retain_all=True, truncate_by_edge=True)

In [7]:
# map each walmart to the nearst road on the map and make sure that the distances (in meter) is small (e.g. less than 300m) 
# or otherwise re-evaluate the graph above with a bigger dist
gta_part['osmid'], gta_part['osmid_dist_m'] = zip(*gta_part.apply(lambda row: ox.nearest_nodes(G, row.longitude, row.latitude, return_dist=True), axis = 1))
gta_part

,latitude,longitude,store_number,phone,address,city,Province,postal_code,osmid,osmid_dist_m
0,43.866158,-79.013226,Walmart Supercentre; #3001,(905) 426-6160,270 Kingston Rd E RR #1,Ajax,ON,L1S 4S7,674536269,277.963737
1,44.015010,-79.411176,Walmart Supercentre; #5778,(905) 841-0300,135 First Commerce Dr,Aurora,ON,L4G 0G2,701911552,246.673809
2,43.871301,-79.215267,Walmart Supercentre; #1109,(905) 472-9582,500 Copper Creek Dr,Markham,ON,L6B 0S1,10291311614,98.517541
3,43.867108,-79.290345,Walmart Supercentre; #3053,(905) 477-6060,5000 Hwy-7 Unit Y006A,Markham,ON,L3R 4M9,437952581,189.265679
4,44.066859,-79.484682,Walmart Supercentre; #3062,(905) 853-8811,17940 Yonge S,Newmarket,ON,L3Y 8S4,699255429,95.426436
5,43.907599,-78.814547,Walmart Supercentre; #1153,(905) 579-3325,1300 King St E,Oshawa,ON,L1H 8J4,1543685682,119.115062
6,43.942780,-78.847706,Walmart Supercentre; #3161,(905) 404-6581,1471 Harmony Rd,Oshawa,ON,L1H 7K5,1768769491,243.984412
7,43.880690,-78.881540,Walmart Supercentre; #1056,(905) 438-1400,680 Laval Dr,Oshawa,ON,L1J 0B5,1360818945,155.728635
8,43.843959,-79.068945,Walmart Supercentre; #3186,(905) 619-9588,1899 Brock Rd Unit #1,Pickering,ON,L1V 4H7,414466395,83.206854
9,43.877256,-79.409737,Walmart Supercentre; #3195,(905) 737-3457,1070 Major Mackenzie Dr E,Richmond Hill,ON,L4S 1P3,414461288,133.692850


In [8]:
# visualize the walmart locations on the map
m = folium.Map(location=gta_part_loc.mean().to_list(), zoom_start=10, scrollWheelZoom=False, dragging=False)
for store_ind in range(wal_gta_count):
    folium.Marker(gta_part_loc.values.tolist()[store_ind],  popup=gta_part['store_number'][store_ind], icon=folium.Icon(color='blue', icon="fa-shopping-cart", prefix='fa'),).add_to(m)
m

In [9]:
# helper function that takes a folium map and plot an antpath based on the route of the osmids 
def draw_route(G, route, m):
    ways_frame = ox.graph_to_gdfs(G)[1]    
    path = []
    for u, v in zip(route[0:], route[1:]):
        try:
            geo = (ways_frame.query(f'u == {u} and v == {v}').to_dict('list')['geometry'])
            m_geo = min(geo,key=lambda x:x.length)
        except:
            geo = (ways_frame.query(f'u == {v} and v == {u}').to_dict('list')['geometry'])
            m_geo = min(geo,key=lambda x:x.length)
        x, y = m_geo.coords.xy
        points = map(list, [*zip([*y],[*x])])
        path.extend([*points][:-1])

    folium.plugins.AntPath(
        locations = path,
        dash_array=[1, 10],
        delay=1000,
        color='red',
        pulse_color='black'
    ).add_to(m)

    return m

In [10]:
# A helper function that takes the graph and it creates a map and draw a path between all locations in it 
def draw_map_path(G, path, locations, routes):
    m = folium.Map(location=locations.mean().to_list(), zoom_start=10, scrollWheelZoom=False, dragging=True)

    route = []

    locationlist = locations.values.tolist()
    num_loc = len(locationlist)
    for i in range(num_loc):
        store_id = path[i]

        folium.Marker(location=locationlist[store_id], icon=folium.Icon(color='white', icon_color='white')).add_to(m)
        icon = folium.DivIcon(
            icon_size=(150,36),
            icon_anchor=(12,40),
            html="""<span class="fa-stack " style="font-size: 12pt" >
                    <span class="fa fa-circle-o fa-stack-2x" style="color : blue"></span>
                    <strong class="fa-stack-1x">
                         {:d}
                    </strong>
                </span>""".format(i)
        )
        folium.Marker(location=locationlist[store_id], icon=icon).add_to(m)
        
        if i == 0:
            store_id_1 = path[num_loc-1]
        else:
            store_id_1 = path[i-1]

        shortest_route = routes[store_id_1][store_id]
        
        if i == num_loc-1:
            route.extend(shortest_route)
        else:
            route.extend(shortest_route[:-1])

    draw_route(G, route, m=m)
        
    return m


In [11]:
# eval the distances between the walmart locations using the graph
gta_part_dists = np.zeros([wal_gta_count, wal_gta_count])
gta_part_pathes = [[[] for i in range(wal_gta_count)] for j in range(wal_gta_count)]
for i in range(wal_gta_count):
    for j in range(wal_gta_count):
        if i==j:
            continue
        gta_part_pathes[i][j] = nx.shortest_path(G=G, source=gta_part.osmid[i], target=gta_part.osmid[j], weight='length')
        gta_part_dists[i][j] = nx.shortest_path_length(G=G, source=gta_part.osmid[i], target=gta_part.osmid[j], weight='length')/1000

In [12]:
# create a TSP object for the problem
gta_part_tsp = TSP(dists=gta_part_dists, gen_method='mutate')

In [13]:
# create an SA object to help solving the tsp problem
sa = SimulatedAnnealing(max_iter=1000, max_iter_per_temp=100, initial_temp=85, final_temp=0.0001, cooling_schedule='linear', cooling_alpha=0.9)

In [14]:
# Get an initial random solution and check its length 
sa.init_annealing(gta_part_tsp)
print(sa.s_cur)
print(sa.val_cur)

# assuming store with index #0 has a warehouse and the tour will start and end with it.
ind = sa.s_cur.index(0)

[0, 2, 3, 7, 11, 4, 12, 14, 1, 17, 13, 8, 15, 9, 16, 10, 5, 6, 0]
573.421903


In [15]:
# Draw the path of the random initial solution
# print(len(sa.s_cur[ind:] + sa.s_cur[:ind]))
draw_map_path(G, sa.s_cur[ind:] + sa.s_cur[:ind], gta_part_loc, gta_part_pathes)

In [17]:
# Run SA and eval the best solution distance
sa.run(gta_part_tsp, repetition=10)
print(sa.val_allbest)
print(gta_part_tsp.eval_solution(sa.s_allbest[ind:] + sa.s_allbest[:ind]))

223.532649
223.532649


In [18]:
# draw the path
draw_map_path(G, sa.s_allbest[ind:] + sa.s_allbest[:ind], gta_part_loc, gta_part_pathes)